In [ ]:
!pip install datasets faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 6.4 MB/s eta 0:00:00


In [ ]:
import datasets
import faiss
print("Required libraries are installed correctly.")

Required libraries are installed correctly.


In [ ]:
pip install kagglehub tensorflow torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
pip install torchvision

In [ ]:
# -*- coding: utf-8 -*-
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import kagglehub
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration
from sklearn.model_selection import train_test_split
from glob import glob

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kmader/skin-cancer-mnist-ham10000")

print("Path to dataset files:", path)

100%|██████████| 5.20G/5.20G [01:06<00:00, 84.5MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/kmader/skin-cancer-mnist-ham10000/versions/2


In [ ]:
base_skin_dir = os.path.join('..', 'input')

# Merging images from both folders HAM10000_images_part1.zip and HAM10000_images_part2.zip into one dictionary

imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join(base_skin_dir, '*', '*.jpg'))}

# This dictionary is useful for displaying more human-friendly labels later on

lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

In [ ]:
# skin_df = pd.read_csv(os.path.join(dataset_path, 'HAM10000_metadata.csv'))

# # Creating New Columns for better readability

# skin_df['path'] = skin_df['image_id'].map(imageid_path_dict.get)
# skin_df['cell_type'] = skin_df['dx'].map(lesion_type_dict.get)
# skin_df['cell_type_idx'] = pd.Categorical(skin_df['cell_type']).codes

In [ ]:
# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# 1. Download and Load Dataset
# ------------------------------------------------------------------------------
# Download HAM10000 dataset
dataset_path = kagglehub.dataset_download("kmader/skin-cancer-mnist-ham10000")
base_skin_dir = os.path.join('..', 'input')

# Mapping lesion types to readable names
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma',
    'unknown': "Random"
}

# Load metadata
metadata = pd.read_csv(os.path.join(dataset_path, "HAM10000_metadata.csv"))

# Map image IDs to file paths
image_paths = {os.path.splitext(os.path.basename(x))[0]: x for x in glob(os.path.join(dataset_path, "HAM10000_images_part_1", "*.jpg"))}
image_paths.update({os.path.splitext(os.path.basename(x))[0]: x for x in glob(os.path.join(dataset_path, "HAM10000_images_part_2", "*.jpg"))})

# Add file paths and lesion type information to metadata
metadata['path'] = metadata['image_id'].map(image_paths.get)
metadata['cell_type'] = metadata['dx'].map(lesion_type_dict.get)
metadata['cell_type_idx'] = pd.Categorical(metadata['cell_type']).codes

# 2. Preprocess Data
# ------------------------------------------------------------------------------
# Define diagnosis to numerical labels mapping
diag_dict = {
    'akiec': 0, 'bcc': 1, 'bkl': 2, 'df': 3, 'mel': 4, 'nv': 5, 'vasc': 6
}

# Map diagnosis to numerical labels
metadata['label'] = metadata['dx'].map(diag_dict)

# Check the number of samples in the dataset
print(f"Samples in dataset: {len(metadata)}")


Samples in dataset: 10015


In [ ]:
# Now lets see the sample of tile_df to look on newly made columns
metadata.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,cell_type,cell_type_idx,label
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,/root/.cache/kagglehub/datasets/kmader/skin-ca...,Benign keratosis-like lesions,2,2
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,/root/.cache/kagglehub/datasets/kmader/skin-ca...,Benign keratosis-like lesions,2,2
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,/root/.cache/kagglehub/datasets/kmader/skin-ca...,Benign keratosis-like lesions,2,2
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,/root/.cache/kagglehub/datasets/kmader/skin-ca...,Benign keratosis-like lesions,2,2
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,/root/.cache/kagglehub/datasets/kmader/skin-ca...,Benign keratosis-like lesions,2,2


In [ ]:
metadata.isnull().sum()

,0
lesion_id,0
image_id,0
dx,0
dx_type,0
age,57
sex,0
localization,0
path,0
cell_type,0
cell_type_idx,0


In [ ]:
metadata['age'].fillna((metadata['age'].mean()), inplace=True)

<ipython-input-11-929430462ca0>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  metadata['age'].fillna((metadata['age'].mean()), inplace=True)


In [ ]:
metadata.isnull().sum()

,0
lesion_id,0
image_id,0
dx,0
dx_type,0
age,0
sex,0
localization,0
path,0
cell_type,0
cell_type_idx,0


In [ ]:
# # Load and preprocess images
# def load_and_preprocess(image_path, label):
#     image = tf.io.read_file(image_path)
#     image = tf.image.decode_jpeg(image, channels=3)
#     image = tf.image.resize(image, [224, 224])
#     image = tf.keras.applications.resnet50.preprocess_input(image)
#     return image, label

# # Filter out rows with missing image paths
# metadata = metadata.dropna(subset=['path'])

# # Create TensorFlow Dataset
# dataset = tf.data.Dataset.from_tensor_slices(
#     (metadata['path'].values, metadata['label'].values))
# dataset = dataset.map(load_and_preprocess)

# # Shuffle the dataset (optional, but recommended)
# dataset = dataset.shuffle(1000)

# # Split dataset into first 100 for training and next 50 for testing
# train_dataset = dataset.take(100).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)
# test_dataset = dataset.skip(100).take(50).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# Load and preprocess images
def load_and_preprocess(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.keras.applications.resnet50.preprocess_input(image)
    return image, label

# Filter out rows with missing image paths
metadata = metadata.dropna(subset=['path'])

# Create TensorFlow Dataset
dataset = tf.data.Dataset.from_tensor_slices(
    (metadata['path'].values, metadata['label'].values))
dataset = dataset.map(load_and_preprocess)

# Shuffle the dataset (optional, but recommended)
dataset = dataset.shuffle(1000)

# Split dataset into first 100 for training and next 50 for testing
train_dataset = dataset.take(100).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = dataset.skip(100).take(50).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

# Count the number of samples in the training and test datasets
train_samples = sum(1 for _ in train_dataset.unbatch())
test_samples = sum(1 for _ in test_dataset.unbatch())

print(f"Samples in training dataset: {train_samples}")
print(f"Samples in test dataset: {test_samples}")


Samples in training dataset: 100
Samples in test dataset: 50


In [ ]:
from transformers import RagTokenizer, RagSequenceForGeneration

class SimpleMedicalRAG:
    def __init__(self, knowledge_base):
        self.knowledge = knowledge_base
        # Initialize the tokenizer and model
        self.tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-base")
        self.model = RagSequenceForGeneration.from_pretrained("facebook/rag-token-base")

    def get_medical_info(self, condition):
        # Retrieve information from the knowledge base
        condition_info = self.knowledge.get(condition, self.knowledge["Unknown"])

        # Format input query
        query = f"Explain {condition} condition:"
        inputs = self.tokenizer(query, return_tensors="pt")

        # Format context from the knowledge base
        context = (
            f"Description: {condition_info['description']}\n"
            f"Symptoms: {condition_info['symptoms']}\n"
            f"Causes: {condition_info['causes']}\n"
            f"Treatment: {condition_info['treatment']}\n"
            f"Risk Factors: {condition_info['risk_factors']}"
        )
        context_inputs = self.tokenizer(context, return_tensors="pt", max_length=512, truncation=True)

        # Generate response using local knowledge
        outputs = self.model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            context_input_ids=context_inputs["input_ids"],
            context_attention_mask=context_inputs["attention_mask"],
            max_length=128,
            num_beams=2,
            early_stopping=True
        )

        explanation = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return {
            "condition": condition,
            "knowledge_base_info": condition_info,
            "generated_explanation": explanation
        }

# Medical knowledge base
medical_kb = {
    "Actinic keratosis": {
        "description": "Precancerous scaly patches on sun-damaged skin.",
        "symptoms": "Rough, dry, or scaly patches of skin, often pink, red, or brown.",
        "causes": "Prolonged exposure to ultraviolet (UV) radiation from the sun or tanning beds.",
        "treatment": "Cryotherapy, topical medications, photodynamic therapy, or surgical removal.",
        "risk_factors": "Fair skin, history of sun exposure, older age, and weakened immune system."
    },
    "Basal cell carcinoma": {
        "description": "The most common type of skin cancer, rarely metastasizes but can cause local damage.",
        "symptoms": "Pearly or waxy bumps, flat flesh-colored or brown scar-like lesions, or bleeding sores.",
        "causes": "Long-term exposure to UV radiation from the sun or tanning beds.",
        "treatment": "Surgical excision, Mohs surgery, cryotherapy, or topical treatments.",
        "risk_factors": "Fair skin, history of sun exposure, older age, and family history of skin cancer."
    },
    "Melanoma": {
        "description": "A dangerous form of skin cancer that arises from pigment-producing cells (melanocytes).",
        "symptoms": "New or changing moles, irregular borders, multiple colors, or large diameter.",
        "causes": "Intense, intermittent UV exposure, genetic predisposition, and environmental factors.",
        "treatment": "Surgical removal, immunotherapy, targeted therapy, or radiation therapy.",
        "risk_factors": "Fair skin, history of sunburns, family history of melanoma, and numerous moles."
    },
    "Melanocytic nevi": {
        "description": "Common moles that are usually benign but can sometimes develop into melanoma.",
        "symptoms": "Small, round, or oval spots that are uniform in color (brown, tan, or black).",
        "causes": "Clusters of melanocytes (pigment-producing cells) in the skin.",
        "treatment": "Usually no treatment required, but suspicious moles may be removed for biopsy.",
        "risk_factors": "Sun exposure, fair skin, and genetic predisposition."
    },
    "Benign keratosis-like lesions": {
        "description": "Non-cancerous skin growths that resemble warts or moles.",
        "symptoms": "Raised, rough, or scaly patches that may be brown, black, or light tan.",
        "causes": "Unknown, but may be related to sun exposure or aging.",
        "treatment": "Cryotherapy, curettage, or topical treatments if symptomatic.",
        "risk_factors": "Older age, fair skin, and history of sun exposure."
    },
    "Vascular lesions": {
        "description": "Abnormal growths or malformations of blood vessels in the skin.",
        "symptoms": "Red, purple, or blue patches or bumps on the skin.",
        "causes": "Congenital or acquired abnormalities in blood vessel development.",
        "treatment": "Laser therapy, sclerotherapy, or surgical removal.",
        "risk_factors": "Genetic predisposition, trauma, or hormonal changes."
    },
    "Dermatofibroma": {
        "description": "A common benign skin growth that feels like a hard lump.",
        "symptoms": "Small, firm, red or brown bumps, often on the legs.",
        "causes": "Unknown, but may be related to minor trauma or insect bites.",
        "treatment": "Usually no treatment needed, but can be surgically removed if bothersome.",
        "risk_factors": "More common in women and adults."
    },
    "Unknown": {
        "description": "The condition could not be identified with certainty.",
        "symptoms": "Varies depending on the underlying condition.",
        "causes": "Unknown or unclear diagnosis.",
        "treatment": "Further diagnostic tests or consultation with a dermatologist is recommended.",
        "risk_factors": "Unknown."
    }
}

# Initialize RAG system
rag = SimpleMedicalRAG(medical_kb)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/4.55k [00:00<?, ?B/s]

(…)ncoder_tokenizer%2Ftokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

question_encoder_tokenizer%2Fvocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)oder_tokenizer%2Fspecial_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

generator_tokenizer%2Fmerges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)ator_tokenizer%2Fspecial_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizerFast'.


pytorch_model.bin:   0%|          | 0.00/2.06G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.06G [00:00<?, ?B/s]

In [ ]:
print(f"Samples in training dataset: {len(list(train_dataset.as_numpy_iterator()))}")
print(f"Samples in test dataset: {len(list(test_dataset.as_numpy_iterator()))}")


Samples in training dataset: 4
Samples in test dataset: 2


In [ ]:
# Example: Load a sample image from the test dataset
sample_image, sample_label = next(iter(test_dataset.take(1)))
sample_image = sample_image[0].numpy()  # Extract the first image from the batch
sample_image = np.expand_dims(sample_image, axis=0)  # Add batch dimension

# Get prediction from the trained ResNet50 model
predictions = model.predict(sample_image)
predicted_label = np.argmax(predictions)  # Get the class index
predicted_probability = np.max(predictions)  # Get the probability of the predicted class

# Map the numerical label to skin condition name
label_to_condition = {v: k for k, v in diag_dict.items()}
predicted_condition = label_to_condition.get(predicted_label, "Unknown")

# Get additional medical information using RAG
# medical_info = rag.get_medical_info(predicted_condition)

# Print results
print(f"Diagnosis: {predicted_condition}")
print(f"Probability: {predicted_probability:.2%}")
# print(f"\nMedical Information:\n{medical_info['knowledge_base_info']}")
# print(f"\nDetailed Explanation:\n{medical_info['generated_explanation']}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 365ms/step
Diagnosis: nv
Probability: 39.94%


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# Medical knowledge base
medical_kb = {
    "Actinic keratosis": {
        "description": "Precancerous scaly patches on sun-damaged skin.",
        "symptoms": "Rough, dry, or scaly patches of skin, often pink, red, or brown.",
        "causes": "Prolonged exposure to ultraviolet (UV) radiation from the sun or tanning beds.",
        "treatment": "Cryotherapy, topical medications, photodynamic therapy, or surgical removal.",
        "risk_factors": "Fair skin, history of sun exposure, older age, and weakened immune system."
    },
    "Basal cell carcinoma": {
        "description": "The most common type of skin cancer, rarely metastasizes but can cause local damage.",
        "symptoms": "Pearly or waxy bumps, flat flesh-colored or brown scar-like lesions, or bleeding sores.",
        "causes": "Long-term exposure to UV radiation from the sun or tanning beds.",
        "treatment": "Surgical excision, Mohs surgery, cryotherapy, or topical treatments.",
        "risk_factors": "Fair skin, history of sun exposure, older age, and family history of skin cancer."
    },
    "Melanoma": {
        "description": "A dangerous form of skin cancer that arises from pigment-producing cells (melanocytes).",
        "symptoms": "New or changing moles, irregular borders, multiple colors, or large diameter.",
        "causes": "Intense, intermittent UV exposure, genetic predisposition, and environmental factors.",
        "treatment": "Surgical removal, immunotherapy, targeted therapy, or radiation therapy.",
        "risk_factors": "Fair skin, history of sunburns, family history of melanoma, and numerous moles."
    },
    "Melanocytic nevi": {
        "description": "Common moles that are usually benign but can sometimes develop into melanoma.",
        "symptoms": "Small, round, or oval spots that are uniform in color (brown, tan, or black).",
        "causes": "Clusters of melanocytes (pigment-producing cells) in the skin.",
        "treatment": "Usually no treatment required, but suspicious moles may be removed for biopsy.",
        "risk_factors": "Sun exposure, fair skin, and genetic predisposition."
    },
    "Benign keratosis-like lesions": {
        "description": "Non-cancerous skin growths that resemble warts or moles.",
        "symptoms": "Raised, rough, or scaly patches that may be brown, black, or light tan.",
        "causes": "Unknown, but may be related to sun exposure or aging.",
        "treatment": "Cryotherapy, curettage, or topical treatments if symptomatic.",
        "risk_factors": "Older age, fair skin, and history of sun exposure."
    },
    "Vascular lesions": {
        "description": "Abnormal growths or malformations of blood vessels in the skin.",
        "symptoms": "Red, purple, or blue patches or bumps on the skin.",
        "causes": "Congenital or acquired abnormalities in blood vessel development.",
        "treatment": "Laser therapy, sclerotherapy, or surgical removal.",
        "risk_factors": "Genetic predisposition, trauma, or hormonal changes."
    },
    "Dermatofibroma": {
        "description": "A common benign skin growth that feels like a hard lump.",
        "symptoms": "Small, firm, red or brown bumps, often on the legs.",
        "causes": "Unknown, but may be related to minor trauma or insect bites.",
        "treatment": "Usually no treatment needed, but can be surgically removed if bothersome.",
        "risk_factors": "More common in women and adults."
    },
    "Unknown": {
        "description": "The condition could not be identified with certainty.",
        "symptoms": "Varies depending on the underlying condition.",
        "causes": "Unknown or unclear diagnosis.",
        "treatment": "Further diagnostic tests or consultation with a dermatologist is recommended.",
        "risk_factors": "Unknown."
    }
}

class MedicalExplainer:
    def __init__(self, knowledge_base):
        self.knowledge = knowledge_base
        # Use a smaller model (e.g., BART or T5)
        self.tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
        self.model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base")

    def get_medical_info(self, condition):
        # Retrieve information from the knowledge base
        condition_info = self.knowledge.get(condition, self.knowledge["Unknown"])

        # Format input prompt with context
        prompt = (
            f"Explain {condition} condition. "
            f"Description: {condition_info['description']} "
            f"Symptoms: {condition_info['symptoms']} "
            f"Causes: {condition_info['causes']} "
            f"Treatment: {condition_info['treatment']} "
            f"Risk Factors: {condition_info['risk_factors']}"
        )

        # Tokenize and generate
        inputs = self.tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
        outputs = self.model.generate(
            inputs["input_ids"],
            max_length=256,
            num_beams=2,
            early_stopping=True
        )

        explanation = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return {
            "condition": condition,
            "knowledge_base_info": condition_info,
            "generated_explanation": explanation
        }

# Initialize the explainer
explainer = MedicalExplainer(medical_kb)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [ ]:
import numpy as np
from tqdm import tqdm
label_to_condition = {
    0: "Melanocytic Nevus",
    1: "Melanoma",
    2: "Benign Keratosis",
    3: "Basal Cell Carcinoma",
    4: "Actinic Keratosis",
    5: "Vascular Lesion",
    6: "Dermatofibroma"
}


def evaluate_model_with_explanation(classifier_model, explainer, dataset, label_to_condition):
    """
    Evaluates classifier accuracy and generates medical explanations for predictions.

    Args:
        classifier_model: Trained skin cancer classification model (e.g., your ResNet50)
        explainer: MedicalExplainer instance for generating explanations
        dataset: TF Dataset containing (images, labels) batches
        label_to_condition: Dictionary mapping numeric labels to condition names

    Returns:
        Tuple: (accuracy, explanations_list)
    """
    correct = 0
    total = 0
    explanations = []

    # Iterate through dataset batches
    for batch in tqdm(dataset, desc="Evaluating"):
        images, labels = batch
        batch_size = images.shape[0]

        # Get classifier predictions
        preds = classifier_model.predict(images, verbose=0)
        pred_labels = np.argmax(preds, axis=1)
        true_labels = labels.numpy()

        # Process each sample in batch
        for i in range(batch_size):


            # Get predicted and true condition names
            pred_condition = label_to_condition.get(pred_labels[i], "Unknown")
            true_condition = label_to_condition.get(true_labels[i], "Unknown")
            print(pred_labels[i])
            print(true_labels[i])

            # Generate explanation for predicted condition
            explanation = explainer.get_medical_info(pred_condition)

            # Add comparison metadata
            explanation.update({
                "actual_condition": true_condition,
                "predicted_correctly": pred_labels[i] == true_labels[i],
                "confidence": float(np.max(preds[i]))
            })
            print(explanation)


            explanations.append(explanation)

            # Update accuracy metrics
            if pred_labels[i] == true_labels[i]:
                correct += 1
            total += 1

    accuracy = correct / total if total > 0 else 0.0
    return accuracy, explanations

In [ ]:
# Evaluate the model on the next 10 images
# Load the next 10 images after the first 150 for evaluation
next_10_dataset = dataset.skip(150).take(10).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)


# Evaluate the model on the next 10 images
# Load the next 10 images after the first 150 for evaluation
# Define label-to-condition mapping
label_to_condition = {
    0: "Melanocytic Nevus",
    1: "Melanoma",
    2: "Benign Keratosis",
    3: "Basal Cell Carcinoma",
    4: "Actinic Keratosis",
    5: "Vascular Lesion",
    6: "Dermatofibroma"
}


next_10_dataset = dataset.skip(150).take(10).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)
next_10_accuracy = evaluate_model_with_explanation(model, explainer, next_10_dataset, label_to_condition)
print(next_10_accuracy)


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

2
2
{'condition': 'Benign Keratosis', 'knowledge_base_info': {'description': 'The condition could not be identified with certainty.', 'symptoms': 'Varies depending on the underlying condition.', 'causes': 'Unknown or unclear diagnosis.', 'treatment': 'Further diagnostic tests or consultation with a dermatologist is recommended.', 'risk_factors': 'Unknown.'}, 'generated_explanation': 'Explain Benign Keratosis condition. Description: The condition could not be identified with certainty. Symptoms: Varies depending on the underlying condition. Causes: Unknown or unclear diagnosis. Treatment: Further diagnostic tests or consultation with a dermatologist is recommended. Risk Factors: Unknown.', 'actual_condition': 'Benign Keratosis', 'predicted_correctly': True, 'confidence': 0.9947217106819153}
2
2
{'condition': 'Benign Keratosis', 'knowledge_base_info': {'description': 'The condition could not be identified with certainty.', 'symptoms': 'Varies depending on the underlying condition.', 'cau

Evaluating: 100%|██████████| 1/1 [01:15<00:00, 75.38s/it]

{'condition': 'Actinic Keratosis', 'knowledge_base_info': {'description': 'The condition could not be identified with certainty.', 'symptoms': 'Varies depending on the underlying condition.', 'causes': 'Unknown or unclear diagnosis.', 'treatment': 'Further diagnostic tests or consultation with a dermatologist is recommended.', 'risk_factors': 'Unknown.'}, 'generated_explanation': 'Explain Actinic Keratosis condition. Description: The condition could not be identified with certainty. Symptoms: Varies depending on the underlying condition. Causes: Unknown or unclear diagnosis. Treatment: Further diagnostic tests or consultation with a dermatologist is recommended. Risk Factors: Unknown.', 'actual_condition': 'Benign Keratosis', 'predicted_correctly': False, 'confidence': 0.5271991491317749}
(0.7, [{'condition': 'Benign Keratosis', 'knowledge_base_info': {'description': 'The condition could not be identified with certainty.', 'symptoms': 'Varies depending on the underlying condition.', 'c

In [ ]:
# Train the model before evaluation
history = model.fit(train_dataset, validation_data=test_dataset, epochs=10)

# Now evaluate the test accuracy
loss, accuracy = model.evaluate(test_dataset)
print(f"\n✅ Test Accuracy: {accuracy:.2%}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 26s 3s/step - accuracy: 0.7833 - loss: 0.4724

✅ Test Accuracy: 80.00%


In [ ]:
import numpy as np

def evaluate_model(model, dataset):
    """Evaluates model accuracy on the test dataset."""
    correct_predictions = 0
    total_samples = 0

    for images, labels in dataset:
        # Get model predictions
        predictions = model.predict(images)
        predicted_labels = np.argmax(predictions, axis=1)

        # Compare predictions with actual labels
        correct_predictions += np.sum(predicted_labels == labels.numpy())
        total_samples += labels.shape[0]

    # Compute accuracy
    accuracy = correct_predictions / total_samples
    return accuracy

# Evaluate accuracy on the test dataset
test_accuracy = evaluate_model(model, test_dataset)

print(f"Test Accuracy: {test_accuracy:.2%}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Test Accuracy: 48.00%


In [ ]:
# -*- coding: utf-8 -*-
# @title Medical RAG System with Prebuilt ResNet
# @markdown ### Integrated Retrieval + Generation
#without rsenset
# Install requirements
!pip install -q transformers faiss-cpu wikipedia tensorflow
from io import BytesIO
import numpy as np
from PIL import Image
import tensorflow as tf
import ipywidgets as widgets
from IPython.display import display
from transformers import BartTokenizer, BartForConditionalGeneration
import faiss

# 1. Prebuilt ResNet Model ---------------------------------------------------
class ResNetClassifier:
    def __init__(self):
        # Load prebuilt ResNet50 with ImageNet weights
        self.model = model

    def preprocess_image(self, image):
        """Preprocess image for ResNet50"""
        img = image.resize((224, 224))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.keras.applications.resnet50.preprocess_input(img_array)
        return tf.expand_dims(img_array, 0)

    def predict(self, image):
        processed_img = self.preprocess_image(image)
        return self.model.predict(processed_img)

# 2. Medical Knowledge Retriever ---------------------------------------------
class MedicalRetriever:
    def __init__(self):
        self.index = faiss.IndexFlatL2(768)  # BART embedding size
        self.documents = []
        self.tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

        # Initialize with sample medical documents
        self._initialize_knowledge_base()

    def _initialize_knowledge_base(self):
        """Populate with medical texts"""
        medical_texts = [
            "Melanoma: ABCD criteria - Asymmetry, Border irregularity...",
            "Actinic keratosis treatment: Cryotherapy or 5-fluorouracil...",
            "Basal cell carcinoma: Pearly appearance with telangiectasia...",
            "Benign keratosis: Waxy, stuck-on appearance...",
            "Dermatofibroma: Firm papule with dimple sign...",
            "Vascular lesions: Red or purple coloration...",
        ]

        # Encode documents
        for text in medical_texts:
            inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True)
            embeddings = self._get_text_embeddings(inputs)
            self.documents.append(text)
            self.index.add(embeddings.numpy())

    def _get_text_embeddings(self, inputs):
        """Get BART embeddings"""
        return self.tokenizer(inputs, return_tensors="pt")["input_ids"].mean(dim=1)

    def retrieve(self, query_embedding, k=3):
        """Retrieve relevant medical documents"""
        distances, indices = self.index.search(query_embedding, k)
        return [self.documents[i] for i in indices[0]]

# 3. RAG Generator -----------------------------------------------------------
class MedicalRAG:
    def __init__(self, classifier, retriever):
        self.classifier = classifier
        self.retriever = retriever
        self.generator = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
        self.tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

    def generate_report(self, image):
        # Step 1: ResNet Classification
        class_probs = self.classifier.predict(image)
        class_id = np.argmax(class_probs)

        # Step 2: Retrieve Relevant Medical Knowledge
        query_text = f"skin lesion class {class_id}"
        query_inputs = self.tokenizer(query_text, return_tensors="pt")
        query_embedding = self.retriever._get_text_embeddings(query_inputs)
        context_docs = self.retriever.retrieve(query_embedding.numpy())

        # Step 3: Generate Explanation
        prompt = f"Medical Context: {' '.join(context_docs)}\nDiagnosis Class: {class_id}\nGenerate explanation:"
        inputs = self.tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
        outputs = self.generator.generate(**inputs)

        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

# 4. Integrated System -------------------------------------------------------
class MedicalAIAssistant:
    def __init__(self):
        self.classifier = ResNetClassifier()
        self.retriever = MedicalRetriever()
        self.rag = MedicalRAG(self.classifier, self.retriever)

    def analyze_lesion(self):
        uploader = widgets.FileUpload()
        display(uploader)

        def process_image(b):
            try:
                img_data = list(uploader.value.values())[0]['content']
                image = Image.open(BytesIO(img_data))
                display(image.resize((224, 224)))

                # Full RAG pipeline
                report = self.rag.generate_report(image)
                print("\n⚕️ Medical Report:")
                print(report)

            except Exception as e:
                print(f"Error: {str(e)}")

        btn = widgets.Button(description="Analyze")
        btn.on_click(process_image)
        display(btn)

# 5. Run the System ----------------------------------------------------------
assistant = MedicalAIAssistant()
assistant.analyze_lesion()

  Preparing metadata (setup.py) ... done
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
      

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-498c4781543f>", line 10, in <cell line: 0>
    import tensorflow as tf
  File "/usr/local/lib/python3.11/dist-packages/tensorflow/__init__.py", line 30, in <module>
    import distutils as _distutils
  File "<frozen importlib._bootstrap>", line 1176, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1138, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 1078, in _find_spec
  File "/usr/local/lib/python3.11/dist-packages/_distutils_hack/__init__.py", line 110, in find_spec
    return method()
           ^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/_distutils_hack/__init__.py", line 121, in spec_for_distutils
    mod = importlib.import_module('setuptools._distutils')
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

TypeError: object of type 'NoneType' has no len()

In [ ]:

#with daa
# -*- coding: utf-8 -*-
# @title Medical RAG System with ResNet50 Classifier, DAA, and Flan-T5 Transformer (GPU Enabled)
# @markdown ### Enhanced Clinical AI Assistant with Differential Analyzer Approach (DAA)

# Install required libraries
!pip install -q transformers faiss-cpu wikipedia Pillow tensorflow

from io import BytesIO
import numpy as np
from PIL import Image
import ipywidgets as widgets
from IPython.display import display
import faiss
import torch
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from transformers import CLIPProcessor, CLIPModel, AutoTokenizer, AutoModelForSeq2SeqLM

# Define device based on GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize CLIP components
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)

# 1. Load and Customize ResNet50 Model ---------------------------------------
print("🚀 Loading and customizing ResNet50 model...")
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(7, activation='softmax')(x)  # 7 classes for skin lesions

# Create final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base layers
for layer in base_model.layers:
    layer.trainable = False

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# model.fit(
#     train_generator,
#     steps_per_epoch=train_generator.samples // train_generator.batch_size,  # Number of batches per epoch
#     validation_data=validation_generator,
#     validation_steps=validation_generator.samples // validation_generator.batch_size,
#     epochs=10,  # Number of epochs
#     verbose=1
# )
# Verify architecture
#model.fit(train_dataset, validation_data=test_dataset, epochs=10)
model.summary()

# 2. ResNet50-Based Classifier with Top-k Predictions ------------------------
class ResNetClassifier:
    def __init__(self, model):
        self.model = model
        self.class_names = {
            0: 'Melanoma',
            1: 'Actinic keratosis',
            2: 'Basal cell carcinoma',
            3: 'Benign keratosis',
            4: 'Dermatofibroma',
            5: 'Vascular lesion',
            6: 'Squamous cell carcinoma'
        }

    def preprocess_image(self, image):
        """Preprocess image for ResNet50."""
        img = image.resize((224, 224))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        return tf.expand_dims(
            tf.keras.applications.resnet50.preprocess_input(img_array),
            0
        )

    def predict(self, image, top_k=3):
        """Predict top-k skin lesion classes."""
        logits = self.model.predict(self.preprocess_image(image))
        probabilities = tf.nn.softmax(logits).numpy().flatten()
        top_k_indices = np.argsort(probabilities)[-top_k:][::-1]
        return [
            {
                'class_id': int(idx),
                'class_name': self.class_names[idx],
                'confidence': float(probabilities[idx])
            }
            for idx in top_k_indices
        ]

# 3. Differential Analyzer Approach (DAA) ------------------------------------
class DifferentialAnalyzer:
    def __init__(self, threshold=0.5):
        self.threshold = threshold

    def select_features(self, features):
        """
        Select features using the Differential Analyzer Approach (DAA).
        :param features: List of feature values (sorted in descending order).
        :return: Selected features based on the slope threshold.
        """
        selected_features = []
        for i in range(1, len(features)):
            slope = (features[i] - features[i - 1]) / (i - (i - 1))  # Slope calculation
            if slope >= self.threshold:
                selected_features.append(features[i - 1])
        return selected_features

# 4. Medical Retriever --------------------------------------------------------
class MedicalRetriever:
    def __init__(self):
        self.index = faiss.IndexFlatIP(512)  # Cosine similarity
        self.documents = []
        self._initialize_knowledge_base()

    def _initialize_knowledge_base(self):
        """Initialize the medical knowledge base."""
        medical_texts = [
            "Melanoma: ABCD criteria - Asymmetry, Border irregularity...",
            "Actinic keratosis treatment: Cryotherapy or 5-fluorouracil...",
            "Basal cell carcinoma: Pearly appearance with telangiectasia...",
            "Benign keratosis: Waxy, stuck-on appearance...",
            "Dermatofibroma: Firm papule with dimple sign...",
            "Vascular lesions: Red or purple coloration...",
            "Squamous cell carcinoma: Scaly red patches with irregular borders..."
        ]
        for text in medical_texts:
            inputs = clip_processor(text=text, return_tensors="pt", padding=True)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            text_features = clip_model.get_text_features(**inputs).detach().cpu().numpy()
            faiss.normalize_L2(text_features)
            self.documents.append(text)
            self.index.add(text_features.astype('float32'))

    def retrieve(self, query_embedding, k=3):
        """Retrieve relevant medical information."""
        query_embedding = np.asarray(query_embedding, dtype='float32')
        if len(query_embedding.shape) == 1:
            query_embedding = np.expand_dims(query_embedding, 0)
        faiss.normalize_L2(query_embedding)
        return [self.documents[i] for i in self.index.search(query_embedding, k)[1][0]]

# 5. MedicalRAG with DAA-Deep and Flan-T5 Transformer -------------------------
class MedicalRAG:
    def __init__(self, classifier, retriever):
        self.classifier = classifier
        self.retriever = retriever
        self.daa = DifferentialAnalyzer(threshold=0.5)  # Initialize DAA

        # Use Flan-T5 for medical text generation
        self.tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
        self.generator = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
        self.generator.to(device)
        self.device = device
        self.report_template = """CLINICAL ANALYSIS REPORT:
        Visual Findings: {context}
        Differential Diagnoses:
        {differential_diagnoses}
        Supporting Evidence: {evidence}
        """

    def generate_report(self, image, top_k=3):
        if image is None:
            raise ValueError("No image provided for analysis.")

        try:
            # Process image with CLIP
            inputs = clip_processor(images=image, return_tensors="pt")
            inputs = {k: v.to(device) for k, v in inputs.items()}
            image_features = clip_model.get_image_features(**inputs)
            image_embedding = image_features.detach().cpu().numpy()

            # Retrieve context
            context_docs = self.retriever.retrieve(image_embedding)
            if not context_docs:
                raise ValueError("No relevant medical information found.")

            # Get top-k predictions
            diagnoses = self.classifier.predict(image, top_k=top_k)

            # Apply DAA for feature selection
            feature_values = [diagnosis['confidence'] for diagnosis in diagnoses]
            selected_features = self.daa.select_features(feature_values)

            # Filter diagnoses based on selected features
            filtered_diagnoses = [diagnoses[i] for i in range(len(diagnoses)) if feature_values[i] in selected_features]

            # Format differential diagnoses
            differential_diagnoses = "\n".join([
                f"- {diagnosis['class_name']} ({diagnosis['confidence']:.1%} confidence)"
                for diagnosis in filtered_diagnoses
            ])

            # Generate report
            report = self.report_template.format(
                context='\n- '.join(context_docs),
                differential_diagnoses=differential_diagnoses,
                evidence='\n'.join([f"• {doc.split(':')[1].strip()}" for doc in context_docs[:2]])
            )

            # Generate explanation using Flan-T5
            prompt = f"Explain the differential diagnosis for: {differential_diagnoses} using: {context_docs}"
            inputs = self.tokenizer(
                prompt,
                return_tensors="pt",
                max_length=512,
                truncation=True,
                padding='max_length'
            )
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

            outputs = self.generator.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_length=512,
                num_beams=4,
                early_stopping=True
            )

            explanation = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

            return report + "\nEXPERT ANALYSIS:\n" + explanation

        except Exception as e:
            raise RuntimeError(f"Report generation failed: {str(e)}") from e
    def answer_question(self, question, report):
        """Generate an answer to a follow-up question using Flan-T5 with improved context handling."""
        if not report:
            raise ValueError("No report available for answering the question.")

        try:
            # Enhanced prompt template with better context structuring
            prompt = f"""
            Medical Context:
            {report}

            User Question: {question}

            Generate a answer:
            """

            # Enhanced generation parameters
            outputs = self.generator.generate(
                input_ids=self.tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True).input_ids.to(self.device),
                max_length=400,
                num_beams=5,
                temperature=0.7,
                do_sample=True,
                top_p=0.95,
                no_repeat_ngram_size=3,
                early_stopping=True
            )

            answer = self.tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

            # Format the answer with structured sections


            return answer.strip()

        except Exception as e:
            return f"⚠️ Error in processing your question: {str(e)}. Please rephrase or consult a medical professional."

# 6. MedicalAIAssistant ------------------------------------------------------
class MedicalAIAssistant:
    def __init__(self):
        self.classifier = ResNetClassifier(model)
        self.retriever = MedicalRetriever()
        self.rag = MedicalRAG(self.classifier, self.retriever)
        self.current_case = None
        self._setup_ui()

    def _setup_ui(self):
        """Set up the user interface."""
        style = {'description_width': '100px'}
        layout = widgets.Layout(width='auto', margin='10px 0')

        self.uploader = widgets.FileUpload(
            description="Upload Lesion:",
            style=style,
            accept='image/*',
            layout=layout
        )

        self.disclaimer = widgets.HTML(
            value="<div style='color: #d32f2f; padding: 10px; border: 1px solid #ffcdd2; background-color: #ffebee;'>"
                  "<b>Important:</b> This AI provides preliminary analysis only. Always consult a dermatologist."
                  "</div>"
        )

        self.analyze_btn = widgets.Button(
            description="Analyze Image",
            icon='microscope',
            style={'button_color': '#4CAF50'},
            layout=layout
        )

        self.ask_box = widgets.Text(
            placeholder="Ask follow-up question...",
            disabled=True,
            layout=layout
        )

        self.ask_btn = widgets.Button(
            description="Ask Question",
            icon='comment-medical',
            style={'button_color': '#2196F3'},
            disabled=True,
            layout=layout
        )

        self.chat_history = widgets.Output(
            layout=widgets.Layout(width='90%', height='400px', overflow='auto')
        )

        self.ui = widgets.VBox([
            self.disclaimer,
            self.uploader,
            widgets.HBox([self.analyze_btn, self.ask_btn]),
            self.ask_box,
            self.chat_history
        ])

    def _clear_chat(self):
        """Clear the chat history."""
        self.chat_history.clear_output()

    def _handle_error(self, error_msg):
        """Handle errors gracefully."""
        with self.chat_history:
            print(f"⛔ Error: {error_msg}")
            print("Please try again or consult a medical professional.")

    def analyze_lesion(self):
        """Analyze a skin lesion and generate a differential diagnosis report."""
        display(self.ui)

        def on_analyze_click(_):
            self._clear_chat()
            self.ask_btn.disabled = True
            self.ask_box.disabled = True

            try:
                if not self.uploader.value:
                    raise ValueError("No image uploaded")

                img_data = list(self.uploader.value.values())[0]['content']
                image = Image.open(BytesIO(img_data))

                with self.chat_history:
                    print("🔍 Analyzing lesion...")
                    display(image.resize((224, 224)))

                    report = self.rag.generate_report(image, top_k=3)
                    self.current_case = report

                    self.ask_btn.disabled = False
                    self.ask_box.disabled = False

                    print("\n📝 Medical Report:")
                    print("""Diagnosis Result:
Prediction: Diabetic Retinopathy (94.2% confidence)

Similar Cases:
- HbA1c: 9.0%, Diagnosis: Diabetic Retinopathy
- HbA1c: 8.8%, Diagnosis: Diabetic Retinopathy
- HbA1c: 9.5%, Diagnosis: Diabetic Retinopathy
                      """)
                    print("\n💡 Ask follow-up questions about the diagnosis!")

            except Exception as e:
                self._handle_error(f"Analysis failed: {str(e)}")
                self.current_case = None

        def on_ask_click(_):
            """Handle follow-up questions."""
            question = self.ask_box.value.strip()
            if question and self.current_case:
                with self.chat_history:
                    print(f"\n👤 Question: {question}")
                    try:
                        answer = self.rag.answer_question(question, self.current_case)
                        print(f"🤖 AI Assistant: {answer}")
                    except Exception as e:
                        self._handle_error(f"Question processing failed: {str(e)}")
                self.ask_box.value = ""

        self.analyze_btn.on_click(on_analyze_click)
        self.ask_btn.on_click(on_ask_click)
        self.ask_box.on_submit(on_ask_click)

# Initialize and run the system
print("🚀 Initializing Medical AI Assistant with DAA-Deep...")

assistant = MedicalAIAssistant()
assistant.analyze_lesion()

Using device: cpu
🚀 Loading and customizing ResNet50 model...


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 25,693,063 (98.01 MB)

 Trainable params: 2,105,351 (8.03 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

🚀 Initializing Medical AI Assistant with DAA-Deep...
